### Pull Data from USGS

https://earthquake.usgs.gov/fdsnws/event/1/

In [ ]:
import pandas as pd
import folium
from folium import plugins
from io import StringIO
import requests

pd.options.display.max_columns = None

In [ ]:
payload = {
    'format': 'csv', 
#     'starttime': None,  # default last 30 days
#     'endtime': '2019-06-03',  # default now
    'minmagnitude': 2.5,  # default null
    'limit': None,  # default null, returns 404 over 20,000
}
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
r = requests.get(url, params=payload)

df = pd.read_csv(StringIO(r.text))
print(df.shape)
df.head()

### Generate Animated Map with `folium`

https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON

In [ ]:
# get faults
r = requests.get('https://raw.githubusercontent.com/'
                 'fraxen/tectonicplates/master/GeoJSON/'
                 'PB2002_boundaries.json')

fault_features = r.json()['features']

In [ ]:
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [r['longitude'], r['latitude']],
        },
        'properties': {
            'time': r['time'][0:-1],
            'popup': (
                f"<strong>Time:</strong> {r['time']}<br>"
                f"<strong>Place:</strong> {r['place']}<br>"
                f"<strong>Magnitude:</strong> {r['mag']} {r['magType']}<br>"
                f"<strong>Depth:</strong> {r['depth']}<br>"
            ),
            'icon': 'circle',
            'iconstyle': {
                'fillOpacity': 0.5,
                'stroke': 0,
                'radius': r['mag'] * 2.5
            },
        }
    } for i, r in df.iterrows()
]

m = folium.Map(
#     location=()
    tiles='CartoDBpositron',
#     zoom_start=1,
#     no_wrap=True,
    min_zoom=1.5,
    max_zoom=5,
    world_copy_jump=True,
)

# add faults
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': fault_features,
    },
    style_function = lambda x: {
        'color': 'red',
        'weight': 0.5,
    }
).add_to(m)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='PT6H', # six hour
    time_slider_drag_update=True,
    duration='PT12H',
    date_options='YYYY-MM-DD HH UTC'
).add_to(m)

folium.plugins.Fullscreen(
    position='topright',
    force_separate_button=True,
).add_to(m)

# m.save('earthquakes.html')
m

### Damage Data
from https://www.ngdc.noaa.gov/nndc/struts/form?t=101650&s=1&d=1

> The Significant Earthquake Database contains information on destructive earthquakes from 2150 B.C. to the present that meet at least one of the following criteria: 
> * Moderate damage (approximately $1 million or more)
> * 10 or more deaths
> * Magnitude 7.5 or greater
> * Modified Mercalli Intensity X or greater
> * the earthquake generated a tsunami

In [ ]:
dmg = pd.read_csv('https://www.ngdc.noaa.gov/nndc/struts/results?'
                  'type_0=Exact&query_0=$ID&t=101650&s=13&d=189&dfn=signif.txt',
                  sep='\t')

dmg.describe()

### Contour Plot

One nice way to display likelihoods of earthquake, once we have them

https://www.tjansson.dk/2018/10/contour-map-in-folium/

In [ ]:
import numpy as np
import pandas as pd
import folium
import branca
from folium import plugins
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage
 
# Setup
temp_mean = 12
temp_std  = 2
debug     = False
 
# Setup colormap
colors = ['#d7191c',  '#fdae61',  '#ffffbf',  '#abdda4',  '#2b83ba']
vmin   = temp_mean - 2 * temp_std
vmax   = temp_mean + 2 * temp_std
levels = len(colors)
cm     = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)
 
# Create a dataframe with fake data
df = pd.DataFrame({
    'longitude':   np.random.normal(11.84,     0.15,     1000),
    'latitude':    np.random.normal(55.55,     0.15,     1000),
    'temperature': np.random.normal(temp_mean, temp_std, 1000)})
 
# The original data
x_orig = np.asarray(df.longitude.tolist())
y_orig = np.asarray(df.latitude.tolist())
z_orig = np.asarray(df.temperature.tolist())
 
# Make a grid
x_arr          = np.linspace(np.min(x_orig), np.max(x_orig), 500)
y_arr          = np.linspace(np.min(y_orig), np.max(y_orig), 500)
x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
 
# Grid the values
z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')
 
# Gaussian filter the grid to make it smoother
sigma = [5, 5]
z_mesh = sp.ndimage.filters.gaussian_filter(z_mesh, sigma, mode='constant')
 
# Create the contour
contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, colors=colors, linestyles='None', vmin=vmin, vmax=vmax)
 
# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=3.0,
    ndigits=5,
    stroke_width=1,
    fill_opacity=0.5)
 
# Set up the folium plot
geomap = folium.Map([df.latitude.mean(), df.longitude.mean()], zoom_start=10, tiles="cartodbpositron")
 
# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    x['properties']['stroke-width'],
        'fillColor': x['properties']['fill'],
        'opacity':   0.6,
    }).add_to(geomap)
 
# Add the colormap to the folium map
cm.caption = 'Temperature'
geomap.add_child(cm)
 
# Fullscreen mode
plugins.Fullscreen(position='topright', force_separate_button=True).add_to(geomap)
 
# Plot the data
# geomap.save(f'data/folium_contour_temperature_map.html'
geomap

### Iteratively Download USGS Data

In [ ]:
from dateutil.parser import parse
import requests
import pandas as pd
from io import StringIO

pd.options.display.max_columns = None

def dl_quake_data(start_date, end_date, page_limt=10000):
    start_date = parse(start_date).isoformat()
    end_date = parse(end_date).isoformat()
    payload = {
        'format': 'csv',
        'starttime': start_date,
        'endtime': end_date,
        'minmagnitude': 2,
        'limit': page_limt,
        'orderby': 'time-asc',
    }
    url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
    r = requests.get(url, params=payload)
    
    if r.status_code != 200:
        print('Error', r.status_code, r.url)
        return False
    
    df = pd.read_csv(StringIO(r.text))
    
    dt_min = df['time'].iloc[0]
    dt_max = df['time'].iloc[-1]
    
    fn = (f'{parse(dt_min).strftime("%Y-%m-%d")}_'
          f'{parse(dt_max).strftime("%Y-%m-%d")}')
    df.to_csv(f'data/{fn}.csv', index=False)
    
    print(fn)
    
    if len(df) == page_limt:
         dl_quake_data(start_date=dt_max,
                       end_date=end_date)
    
    return True

In [ ]:
# done '1999-01-01' to '2019-01-01'
# dl_quake_data('1999-01-01', '2009-01-01', 10000)

In [ ]:
from pathlib import Path

dfs = []
for csv in Path('data').iterdir():
    dfs.append(pd.read_csv(csv))
    
df = pd.concat(dfs)
df['time'] = pd.to_datetime(df['time'])
df['updated'] = pd.to_datetime(df['updated'])

In [ ]:
df['mag'].hist(bins=int((df.mag.max() - df.mag.min()) * 10));

In [ ]:
(df['updated'] - df['time']).dt.days.hist(bins=50);

In [ ]:
df.groupby(df['time'].dt.year)['id'].count().plot();

### Group Earthquake Events to Region

This is done because different regions have different detective power for small magnitude eathquakes. Here I generate a grid of equi-distant points around the globe before clustering earthquake events to the nearest node.

As improvements, I could define these regions algorithmically (like with [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/index.html), [clusterpy](https://github.com/clusterpy/), [Moran'sI](https://en.wikipedia.org/wiki/Moran's_I)) based on earthquake characteristics in each regions. It may be that a special model already exists for this purpose [in extant research](https://www.researchgate.net/publication/260702383_A_detailed_seismic_zonation_model_for_shallow_earthquakes_in_the_broader_Aegean_area). Alternatively, knowing where seismic stations are located and how sensitive they are could help inform region definitions.

In [ ]:
import hdbscan

downsamp = df[['latitude', 'longitude']].sample(frac=0.125)
clusterer = hdbscan.HDBSCAN(metric='haversine')
clusterer.fit(downsamp)

In [ ]:
# roughly equidistant points on a unit-radius cartesian sphere
#https://stackoverflow.com/a/44164075/11208892
import numpy as np
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt

num_pts = 3000
indices = np.arange(0, num_pts, dtype=float) + 0.5

phi = np.arccos(1 - 2*indices/num_pts)
theta = np.pi * (1 + 5**0.5) * indices

x = np.cos(theta) * np.sin(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(phi)

plt.figure().add_subplot(111, projection='3d').scatter(x, y, z);

In [ ]:
# convert to lat/lon, ignoring that the earth is not a sphere
# https://stackoverflow.com/questions/1185408/converting-from-longitude-latitude-to-cartesian-coordinates
clust_df = pd.DataFrame({'clust_lat': 180 * np.arcsin(z / 1) / np.pi, 
                         'clust_lon': 180 * np.arctan2(y, x) / np.pi})
clust_df['clust_id'] = clust_df.index

### Assign Quake Events to Cluster Centers

In [ ]:
from sklearn.neighbors import DistanceMetric

dist = DistanceMetric.get_metric('haversine') # requires radian inputs

clust_rads = clust_df[['clust_lat', 'clust_lon']].values * np.pi / 180
quake_rads = df[['latitude', 'longitude']].values * np.pi / 180

nearest_centers = []
step = 10000
i = 0
while i < len(quake_rads):
    print(round(i/len(quake_rads))*100, '%')
    dists = dist.pairwise(quake_rads[i:i+step], clust_rads)
    indices = np.argmin(dists, axis=1)  # indices of nearest cluster
    nearest_centers.append(clust_rads[indices])
    i += step
    
nearest_centers = np.vstack(nearest_centers)

In [ ]:
df['clust_lat'], df['clust_lon'] = np.nan, np.nan
df[['clust_lat', 'clust_lon']] = 180 * nearest_centers / np.pi

In [ ]:
df = df.merge(clust_df, on=['clust_lat', 'clust_lon'])

### Filter to clusters with >= 2000 observations

In [ ]:
clust_cnts = df.groupby(['clust_id']).size()
clust_cnts = clust_cnts.loc[clust_cnts >= 2000]

clust_df = clust_df.loc[clust_df['clust_id'].isin(clust_cnts.index)]
df = df.loc[df['clust_id'].isin(clust_cnts.index)]

In [ ]:
clust_cnts.shape

### Map Cluster Sizes

In [ ]:
import folium
import folium.plugins

m = folium.Map(tiles='CartoDBpositron')
clust_cnts = df.groupby(['clust_id', 'clust_lat', 'clust_lon']).size()

for i, v in clust_cnts.items():
    folium.CircleMarker(
        i[1:], 
        radius=np.sqrt((v/100) / np.pi) * 2,
        fill=True,
        fill_opacity=0.5,
        stroke=0,
        popup=f"id:{i[0]}<br>events: {v}",
    ).add_to(m)

folium.plugins.Fullscreen(
    position='topright',
    force_separate_button=True,
).add_to(m)

m

### Identify Cut-Off Magnitude for Each Node, `M_c`

Here I compare OLS and Mode methods. For a more robust method, see [Bayesian method](https://medium.com/the-history-risk-forecast-of-perils/exploring-the-fascinating-world-of-incomplete-seismicity-data-part-i-ii-bayesian-inference-386338b43b71).

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def OLS_mc_a_b(node_df):
    node_df = node_df.copy()
    node_df['mag'] = node_df['mag'].round(1)
    node_df = node_df.loc[node_df['mag']<6]
    
    n_steps = int((node_df['mag'].max() - node_df['mag'].min()) * 10) + 1
    mag_range = np.linspace(
        node_df['mag'].min(), 
        node_df['mag'].max(), 
        n_steps)
    mag_range = [round(x, 2) for x in mag_range]

    counts = pd.Series(
        index=mag_range,
        data=[sum(node_df['mag'] >= x) for x in mag_range]
    )

    best_score = 0
    best_m_c, a, b = [None] * 3

    for M_c in np.arange(node_df['mag'].min(), 4.6, 0.1):
        M_c = round(M_c, 2)
        data = counts.loc[counts.index>=M_c]
        X = np.array(data.index).reshape(-1, 1)
        y = np.log10(data.values)
        reg = LinearRegression().fit(X, y)
        score = reg.score(X, y)

        if score > best_score:
            best_m_c = M_c
            best_score = score
            a = reg.intercept_
            b = -1 * reg.coef_[0]

    return best_m_c, a, b


def mode_m_c_a_b(node_df):
    node_df = node_df.copy()
    node_df['mag'] = node_df['mag'].round(1)
    node_df = node_df.loc[node_df['mag']<6]
    m_c = node_df['mag'].mode()[0]
    node_df = node_df.loc[node_df['mag']>=m_c]
    a, b, _ = fit_GR_OLS(node_df)
    
    return m_c, a, b


def fit_GR_OLS(node_df):
    n_steps = int((node_df['mag'].max() - node_df['mag'].min()) * 10) + 1
    mag_range = np.linspace(
        node_df['mag'].min(), 
        node_df['mag'].max(), 
        n_steps)
    mag_range = [round(x, 2) for x in mag_range]

    counts = pd.Series(
        index=mag_range,
        data=[sum(node_df['mag'] >= x) for x in mag_range]
    )

    X = np.array(counts.index).reshape(-1, 1)
    y = np.log10(counts.values)
    reg = LinearRegression().fit(X, y)
    a = reg.intercept_
    b = -1 * reg.coef_[0]
    mse = mean_squared_error(y, reg.predict(X))
    
    return a, b, mse

In [ ]:
clust_id = 609
node_df = df.loc[df['clust_id']==clust_id]

In [ ]:
# OLS method
m_c, a, b = OLS_mc_a_b(node_df)

fig, ax = plt.subplots()
node_df['mag'].hist(bins=50, ax=ax)
plt.axvline(x=m_c, linewidth=2, color='r');

In [ ]:
# mode method
m_c, a, b = mode_m_c_a_b(node_df)

fig, ax = plt.subplots()
node_df['mag'].hist(bins=50, ax=ax)
plt.axvline(x=m_c, linewidth=2, color='r');

In [ ]:
# apply mode method
clust_df['m_c'], clust_df['a'], clust_df['b'] = np.nan, np.nan, np.nan

for clust_id in df['clust_id'].unique():
    node_df = df.loc[df['clust_id']==clust_id]
    m_c, a, b = mode_m_c_a_b(node_df)
    clust_df.loc[clust_df['clust_id']==clust_id, ['m_c', 'a', 'b']] = m_c, a, b

In [ ]:
clust_df['m_c'].hist();

### Filter Earthquake events by `m_c`

In [ ]:
df = df.merge(clust_df[['clust_id', 'm_c', 'a', 'b']], on='clust_id')

In [ ]:
df = df.query("mag >= m_c")

### Timeseries Feature Engineering

#### How Many Samples?

In literature, 50 of the most recent quakes were analyzed when the magnitude cutoff was 4.0. However, some regions have a lower threshold, allowing for exponentially more events. Therefore, I will take exponentially more events when calculated features for those regions:

In [ ]:
def get_n_samples(m_c):
    # From research:
    #  m_c=4, y=50
    # Extrapolated to the mc=0 case, with exponential increase
    #  m_c=0, y=500000
    # log10(n_events) = slope * m_c + y_int
    y_int = np.log10(500000)
    slope = -1 # by definition of logarithm

    return int(round(10 ** (slope * m_c + y_int)))

In [ ]:
# slolam, solstice, teksystems global services

In [ ]:
# at weekly interval, collect past 100 events, predict whether 5.5+ will occur in the next 30 days

strt_sunday = df['time'].min() - pd.to_timedelta(df['time'].min().dayofweek, unit='d')
end_sunday = (df['time'].max() - pd.to_timedelta(df['time'].max().dayofweek, unit='d') - 
              pd.to_timedelta(30, unit='d')) # leave 30 day buffer at end to assess whether quake occured
sampling_periods = pd.date_range(strt_sunday, end_sunday, freq='W')

x_df = pd.DataFrame(columns=['clust_id','sample_date',
                             'T_days','M_mean','dEsq',
                             'a_lsq','b_lsq','a_mlk', 'b_mlk','mse',
                             'diff_M_max_obs_exp','mu','c',
                             'quake_next30'])

for node_id, node_alltime_df in df.loc[df['clust_id']==378].groupby(['clust_id']):
    print(node_id)
    
    m_c = clust_df.loc[clust_df['clust_id']==node_id, 'm_c'].values[0]
    N_events = get_n_samples(m_c)
    
    for sample_date in sampling_periods:
        print(sample_date) #2007-01-14 02:03:26.520000+00:00
        # check if 5.5+ mag quake happend in next 30 days
        condition = ((sample_date < node_alltime_df['time']) & 
                     (node_alltime_df['time'] < (sample_date + pd.to_timedelta(30, unit='d'))))
        quake_next30 = sum(node_alltime_df.loc[condition, 'mag'] >= 5.5) > 0
        
        # take last 100 events, compute features
        node_df = node_alltime_df.loc[node_alltime_df['time']<=sample_date]
        
        if len(node_df) < N_events:
            continue
        
        node_df = node_df[-N_events:]

        # T: time period in days
        T = (node_df['time'].max() - node_df['time'].min()).days

        # M_mean: mean Magnitude
        M_mean = node_df['mag'].mean()

        # dEsq: seismic energy release
        dEsq = np.sum(
            np.sqrt(
                np.power(
                    np.array([10]*len(node_df)), 
                    11.8+1.5*node_df['mag']
                )
            )
        ) / T

        # a and b values (2 methods) and MSE from GR law
        n = len(node_df)
        # some issues in here:
        a_lsq, b_lsq, mse = fit_GR_OLS(node_df)
        
        b_mlk = np.log10(np.e) / (node_df['mag'].mean() - node_df['mag'].min())
        a_mlk = np.log10(n) + b_mlk * node_df['mag'].min()
                
        # difference between the maximum observed and the maximum expected
        diff_M_max_obs_exp = node_df['mag'].max() - a_lsq / b_lsq
        
        # mean and stdev time between mag 4.5 & 5 events
        diffs = node_df.loc[(4.5 <= node_df['mag']) & (node_df['mag'] <= 5), 'time'].diff()

        mu = diffs.mean().total_seconds()
        c = diffs.std().total_seconds() / mu
  
        # Maximum magnitude in last seven days
        # date_7days = node_df['time'].max() - pd.to_timedelta(7, unit='d')
        # x_6i = node_df.loc[node_df['time'] > date_7days, 'mag'].max()

        x_df = x_df.append(pd.DataFrame({
            'clust_id': [node_id],
            'sample_date': [sample_date],
            'T_days': [T],
            'M_mean': [M_mean],
            'dEsq': [dEsq],
            'a_lsq': [a_lsq],
            'b_lsq': [b_lsq],
            'a_mlk': [a_mlk],
            'b_mlk': [b_mlk],
            'mse':[mse],
            'diff_M_max_obs_exp': [diff_M_max_obs_exp],
            'mu': [mu],
            'c': [c],
            'quake_next30': [quake_next30],
        }))

In [ ]:
x_df.groupby(['clust_id']).size()

In [ ]:
x_df.groupby(['clust_id']).size()

In [ ]:
df.loc[df['clust_id']==node_id, 'time'].max()

In [ ]:
len(df.loc[df['clust_id']==node_id])

In [ ]:
df.loc[df['clust_id']==node_id, 'mag'].hist()